# Introduction

In this notebook we preprocess a data set of Google Play Store applications and their relevant information. The data set can be found [here](https://www.kaggle.com/lava18/google-play-store-apps)

In [489]:
import pandas as pd
df = pd.read_csv('googleplaystore.csv')

# Dataset exploration and modification

## Data types

The data set has 13 columns.

App, Category, Type, Content Rating, Genres, Current Ver, and Android Ver are categorical data types

Rating, Reviews, Size, Installs, Price, and Last Updated are numerical data types.

Some columns such as Size and Installs are alphanumeric (e.g. 100k, 200M, 10000+)

From the numeric data, Rating and Price are continual and the rest are discrete

In [490]:
len(df.columns)

13

In [491]:
df.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

In [492]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


## Data quality

Analyzing the data set we can see there are several problems. 

First of all we notice that the max rating is listed as 19. This is a mistake in the data set as rating can only go up to 5. This is caused by one row that has its columns shifted to the left.

In [493]:
df.describe(include = 'all')

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
count,10841,10841,9367.000000,10841,10841,10841,10840,10841,10840,10841,10841,10833,10838
unique,9660,34,NaN,6002,462,22,3,93,6,120,1378,2832,33
top,ROBLOX,FAMILY,NaN,0,Varies with device,"1,000,000+",Free,0,Everyone,Tools,"August 3, 2018",Varies with device,4.1 and up
freq,9,1972,NaN,596,1695,1579,10039,10040,8714,842,326,1459,2451
mean,NaN,NaN,4.193338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,0.537431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,4.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,4.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [494]:
df.loc[df['Rating'] == 19]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [495]:
df.drop(index=10472, inplace=True)

Next, we notice a lot of duplicates. These need to be removed.

Apart from completely identical rows, there are some duplicate rows where only the reviews differ. We keep the ones with the most reviews as it is more likely that's the most recent version.

In [496]:
df.sort_values('Reviews', ascending=False, inplace=True)
df.drop_duplicates(subset='App', keep='first', inplace=True)
df.sort_index(inplace=True)

## Missing values

There are a lot of missing values for the Rating column. Unfortunately we cannot estimate or replace these values so we will have to remove them.

In [497]:
df.isnull().sum()

App                  0
Category             0
Rating            1463
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       0
Genres               0
Last Updated         0
Current Ver          8
Android Ver          2
dtype: int64

In [498]:
df.dropna(subset=['Rating', 'Type', 'Current Ver', 'Android Ver'], inplace=True)

In [499]:
df.isnull().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       0
Android Ver       0
dtype: int64

In [500]:
df.describe(include="all")

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
count,8190,8190,8190.000000,8190,8190,8190,8190,8190,8190,8190,8190,8190,8190
unique,8190,33,NaN,5319,413,19,2,73,6,115,1299,2625,31
top,BN Pro White Text,FAMILY,NaN,2,Varies with device,"1,000,000+",Free,0,Everyone,Tools,"August 3, 2018",Varies with device,4.1 and up
freq,1,1651,NaN,82,1169,1415,7588,7588,6612,717,245,1011,1812
mean,NaN,NaN,4.173162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,0.536589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,4.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,4.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sampling

Sampling is often used for two reasons:
- Collecting all the data is too expensive or time consuming
- Analysing the data takes too much computational power

Point one does not apply here, the data has already been collected. Point two can also be ignored because our data set is relatively small, meaning there is no need to sample it

## Dimensionality reduction

### Feature subset selection

There is plenty of irrelevant information in our data set. App name is only an identifier and not important. Current version also tells nothing about the application. We can remove these columns from the data set.

In [501]:
df.drop(columns=['App', 'Last Updated', 'Current Ver', 'Android Ver'], inplace=True)

## Transformation

The Size column has values in both kilobytes and megabytes. We normalize these to kilobytes and remove the postfix.

We also remove the '$' prefix from the Price column.

In [502]:
def to_kilobytes(x):
  last_char = x[len(x) - 1]
  value = x[:len(x) - 1]
  if (last_char == 'k'):
    return float(value)
  elif (last_char == 'M'):
    return float(value) * 1000

def remove_dollar(x):
  if (x[0] == '$'):
    return float(x[1:])
  return float(x)

df['Size'] = df['Size'].apply(to_kilobytes)
df['Price'] = df['Price'].apply(remove_dollar)

## Feature creation

Apps often have two genres. These are delimited with ';'. New columns can be created to represent a primary genre and a secondary genre.

In [503]:
df[['Genre 1', 'Genre 2']] = df['Genres'].str.split(';', expand=True)
df.drop(columns=['Genres'], inplace=True)

## Binarization

In our dataset there were only two values in the column Type, so we changed the name to IsFree and binarized the values

In [504]:
df.loc[df['Type'] == 'Free', 'Type'] = 1
df.loc[df['Type'] == 'Paid', 'Type'] = 0
df.rename(columns={'Type': 'IsFree'}, inplace=True)

Here is a sample of the preprocessed data set

In [508]:
df.sample(n=30)

,Category,Rating,Reviews,Size,Installs,IsFree,Price,Content Rating,Genre 1,Genre 2
2067,FAMILY,4.5,19230,45000.0,"5,000,000+",1,0.00,Everyone,Educational,Brain Games
9494,TRAVEL_AND_LOCAL,4.4,22776,55000.0,"1,000,000+",1,0.00,Everyone,Travel & Local,None
4941,TOOLS,3.7,9514,7000.0,"500,000+",1,0.00,Everyone,Tools,None
8743,LIFESTYLE,3.9,1638,4900.0,"100,000+",1,0.00,Everyone,Lifestyle,None
5599,FAMILY,4.4,228130,70000.0,"5,000,000+",1,0.00,Everyone 10+,Strategy,None
8146,BOOKS_AND_REFERENCE,5.0,2,1400.0,5+,1,0.00,Everyone,Books & Reference,None
10228,PHOTOGRAPHY,4.7,37,10000.0,"1,000+",1,0.00,Everyone,Photography,None
8368,TOOLS,3.0,19,22000.0,"1,000+",0,4.99,Everyone,Tools,None
7351,SHOPPING,4.4,9950,15000.0,"500,000+",1,0.00,Everyone,Shopping,None
8609,NEWS_AND_MAGAZINES,2.6,820,NaN,"100,000+",1,0.00,Everyone,News & Magazines,None
